In [ ]:
# Build a ChatBot class with all necessary modules to make a complete conversation
class ChatBot():
    # initialize
    def __init__(self):
        # once chat starts, the history will be stored for chat continuity
        self.chat_history_ids = None
        # make input ids global to use them anywhere within the object
        self.bot_input_ids = None
        # a flag to check whether to end the conversation
        self.end_chat = False
        
    def user_input(self):
        self.end_chat = False
        # receive input from user
        text = input_widget.value
        input_widget.value = ""
        output_widget.value += f"<p><strong>You:</strong> {text}</p>"
        text = text.lower()
        # end conversation if user wishes so
        if text.strip() in ['bye', 'quit', 'exit']:
            # turn flag on 
            self.end_chat = True
            # a closing comment
            output_widget.value += f"<p><strong>ChatBot:</strong> See you soon! Bye!</p>"
        else:
            # continue chat, preprocess input text
            # encode the new user input, add the eos_token and return a tensor in Pytorch
            self.new_user_input_ids = tokenizer.encode(text + tokenizer.eos_token, \
                                                       return_tensors='pt')

    def bot_response(self):
        if self.end_chat == False:
            # append the new user input tokens to the chat history
            # if chat has already begun
            if self.chat_history_ids is not None:
                self.bot_input_ids = torch.cat([self.chat_history_ids, self.new_user_input_ids], dim=-1) 
            else:
                # if first entry, initialize bot_input_ids
                self.bot_input_ids = self.new_user_input_ids

            # define the new chat_history_ids based on the preceding chats
            # generated a response while limiting the total chat history to 1000 tokens, 
            self.chat_history_ids = model.generate(self.bot_input_ids, max_length=1000, \
                                                   pad_token_id=tokenizer.eos_token_id)

            # last ouput tokens from bot
            response = tokenizer.decode(self.chat_history_ids[:, self.bot_input_ids.shape[-1]:][0], \
                                   skip_special_tokens=True)
            # in case, bot fails to answer
            if response == "":
                response = self.random_response()
            # print bot response
            output_widget.value += f"<p><strong>ChatBot:</strong> {response}</p>"
        
    # in case there is no response from model
    def random_response(self):
        i = -1
        response = tokenizer.decode(self.chat_history_ids[:, self.bot_input_ids.shape[i]:][0], \
                               skip_special_tokens=True)
        # iterate over history backwards to find the last token
        while response == '':
            i = i-1
            response = tokenizer.decode(self.chat_history_ids[:, self.bot_input_ids.shape[i]:][0], \
                               skip_special_tokens=True)
        # if it is a question, answer suitably
        if response.strip() == '?':
            reply = np.random.choice(["I don't know", 
                                     "I am not sure"])
        # not a question? answer suitably
        else:
            reply = np.random.choice(["Great", 
                                      "Fine. What's up?", 
                                      "Okay"
                                     ])
        return reply